In [3]:
import glob 
import json
import os
print(os.getcwd())

/nfs/zhujinguo/datasets/visual_genome/annotations


In [4]:
# 5.41M caption for "region_descriptions.json"

In [5]:
with open(f"region_descriptions.json", "r") as fp: #5.41
    captions = json.load(fp)

In [6]:
print(captions[1].keys())

dict_keys(['regions', 'id'])


In [7]:
captions[1]['id']

2

In [90]:
captions[1]['regions'][:2]


[{'region_id': 1387,
  'width': 43,
  'height': 17,
  'image_id': 2,
  'phrase': 'walk sign is lit up',
  'y': 193,
  'x': 465},
 {'region_id': 1388,
  'width': 133,
  'height': 253,
  'image_id': 2,
  'phrase': 'man wearing silver backpack',
  'y': 322,
  'x': 331}]

In [91]:
print(len(captions[1]['regions']))

256


In [92]:
captions[1]['regions'][0]

{'region_id': 1387,
 'width': 43,
 'height': 17,
 'image_id': 2,
 'phrase': 'walk sign is lit up',
 'y': 193,
 'x': 465}

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 108077/108077 [00:01<00:00, 58748.17it/s]


In [93]:
from collections import defaultdict
from tqdm import tqdm
iid2captions = defaultdict(set)
for cap in tqdm(captions):
    cap = cap["regions"]
    
    for c in cap:
        # v0
        # iid2captions[c["image_id"]].append(c['phrase'])
        region_area = int(c['height'])*int(c['width'])
        if region_area >= 128*128:
            iid2captions[c["image_id"]].add(c['phrase'])


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 108077/108077 [00:03<00:00, 34841.72it/s]


In [109]:
for iid in iid2captions.keys():
    iid2captions[iid] = list(iid2captions[iid])

In [110]:
len(iid2captions)

107823

In [ ]:
iid2captions[1]

In [111]:
import random
from glob import glob
paths = list(glob(f"../images/VG_100K/*.jpg")) + list(
    glob(f"../images/VG_100K_2/*.jpg")
)
random.shuffle(paths)
caption_paths = [
    path for path in paths if int(path.split("/")[-1][:-4]) in iid2captions
]
iid2subset = {}
for path in paths:
    if int(path.split("/")[-1][:-4]) in iid2captions:
        iid2subset[int(path.split("/")[-1][:-4])] = os.path.join(path.split("/")[-2],path.split("/")[-1])
    

if len(paths) == len(caption_paths):
    print("all images have caption annotations")
else:
    print("not all images have caption annotations")
print(
    len(paths), len(caption_paths), len(iid2captions), len(iid2subset)
)


not all images have caption annotations
108249 107823 107823 107823


In [ ]:
paths


In [112]:
num=0
for iid in iid2captions.keys():
    num += len(iid2captions[iid])
print(num)

1645544


25614848

In [107]:
iid2subset[1]

'VG_100K_2/1.jpg'

In [113]:

data = {
    "phrase":  iid2captions,
    "subset": iid2subset,
}
json.dump(data, open("vg_captions_128filter.json", "w"))


